In [15]:
import json
from datasets import load_dataset

ds = load_dataset("Aoschu/German_invoices_dataset_for_donut")["train"]

In [16]:
def load_sample(sample):
    return sample["image"], json.loads(sample["ground_truth"])["gt_parse"]["transcription"]

img, transcriptions = load_sample(ds[0])

In [17]:
import asyncio
import openai
import instructor
from pydantic import BaseModel
import weave
import io, base64
from typing import Union
from PIL import Image

weave.init("german_invoices_eval")

client = instructor.from_openai(openai.AsyncOpenAI())

class QuestionAnswer(BaseModel):
    question: str
    answer: str

def image_to_base64(image_path: Union[str, Image]) -> str:
    image = Image.open(image_path) if isinstance(image_path, str) else image_path
    byte_arr = io.BytesIO()
    image.save(byte_arr, format="PNG")
    encoded_string = base64.b64encode(byte_arr.getvalue()).decode("utf-8")
    return str(encoded_string)

@weave.op
async def create_question_answer_pairs(image: Image, transcriptions: list[str]) -> list[QuestionAnswer]:
    "Call openai to create question answer pairs"
    image_base64 = image_to_base64(image)
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that creates question answer pairs from a given text. The text is a list of OCR output from a german invoice data.",
        },
        {
            "role": "user",
            "content": [
                {
                    "type":"text",
                    "text": f"Here are the different OCR extractions from the document: {transcriptions}. \
                        Create 5 question answer pairs based on the image and the extracted text. \
                        Write the questions and answers in German."
                },  
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}",
                    },
                },
            ],
        },
    ]
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        response_model=list[QuestionAnswer],
    )
    return response

In [18]:
qa_pairs = await create_question_answer_pairs(img, transcriptions)
qa_pairs

🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-80d2-76e1-a4b2-8f01b8038a1b


[QuestionAnswer(question='Wer ist der Anbieter des Angebots?', answer='Paul Cheung'),
 QuestionAnswer(question='Auf welches Datum ist das Angebot datiert?', answer='29.07.2030'),
 QuestionAnswer(question='Was ist die Angebotsnummer?', answer='1234'),
 QuestionAnswer(question='Wie lautet die Kundenadresse?', answer='Frau Mia Hobner, Zwinglistr. 67, 54321 Münster'),
 QuestionAnswer(question='Was ist der Gesamtbetrag des Angebots?', answer='7.735,00 €')]

In [19]:
from tqdm.asyncio import tqdm_asyncio

async def process_sample(sample):
    img, transcriptions = load_sample(sample)
    qa_pairs = await create_question_answer_pairs(img, transcriptions)
    return {"image": img, "qa_pairs": qa_pairs}

async def process_all_samples():
    tasks = [process_sample(sample) for sample in ds]
    qa_ds = await tqdm_asyncio.gather(*tasks, desc="Processing samples")
    return qa_ds

qa_ds = await process_all_samples()

🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-cdc1-7782-9c54-039ab69d51f1
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-a516-77d1-9647-07b1a08b262c
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-9d0d-7303-8ded-7997480932f0


🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-ad53-75e0-86f2-c43adb3564e7
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-9cd1-79a3-bbce-bde13e0273af
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-ca57-7871-bed6-0411aea81d67
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-b2fa-7081-a17b-2850425cb6f8
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-9c03-7ea3-a753-dac2cc7315be
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-a1df-7520-9a0e-325493703344


🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-ca13-7bc2-83d5-9b3503d80349
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-a0c0-70e3-a782-b9bfc8aaef62
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-97f4-7612-8ad7-90c78b8d75e3


🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-97c9-77a1-bfd5-be79e05addf0
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-a5f5-78f0-8fc0-1b6388d05aa6
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-cb60-7f41-b709-6f40cc3e4592
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-b821-7372-9d36-c9ec57c298b2
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-a2a6-7021-b31e-4c65c869473b
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-9dc7-70c3-8d94-08ef42abfd47


🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-a476-7b63-9c38-96ffb4148adf
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-cc20-7970-a4f5-06d1d7ecb707
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-cc8b-7840-a1d8-abb817be22ec
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-9e25-7923-8ee8-bd178e6ae97f
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-982f-75f3-8b91-ef0106949e74
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-a3d0-7ec0-aba8-ac15666978bb


...

🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-b1e0-7801-a0bd-fca7f6d72b82
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-975e-7b80-ab5e-bc9a0054d600
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-aab3-72d0-862b-f5095ec5c9ad


🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-b919-7f41-9ef5-431c46516fec
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-c729-7b40-8190-c17a6b741ef2


🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-bf2e-7230-a5c9-f19557107cfa
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-bfee-7e53-9dae-3f4e0339579e
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-bce5-7202-b311-b8841282b15d


Processing samples: 100%|██████████| 97/97 [00:24<00:00,  3.93it/s]

🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-c1d8-7640-af8a-a088afa9eb98
🍩 https://wandb.ai/capecape/german_invoices_eval/r/call/0192335b-9185-7c73-ae66-ef6ca0f90427


In [20]:
qa_ds_list = [{"image": sample["image"], 
  "qa_pairs": [{"question": qa_pair.question, 
                "answer": qa_pair.answer} for qa_pair in sample["qa_pairs"]]} 
 for sample in qa_ds]

In [21]:
# save to hf datasets
from datasets import Dataset

qa_ds = Dataset.from_list(qa_ds_list)

qa_ds.save_to_disk("qa_german_invoices_formatted")

Saving the dataset (1/1 shards): 100%|██████████| 97/97 [00:00<00:00, 6077.70 examples/s]


In [22]:
from datasets import load_from_disk
ds = load_from_disk("qa_german_invoices_formatted")

In [23]:
ds[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1414x2000>,
 'qa_pairs': [{'answer': 'Paul Cheung',
   'question': 'Wer ist der Absender des Angebots?'},
  {'answer': 'Frau Mia Hobner',
   'question': 'An wen ist das Angebot gerichtet?'},
  {'answer': '1234', 'question': 'Was ist die Angebotsnummer?'},
  {'answer': '7.735,00 €',
   'question': 'Wie hoch ist der Gesamtbetrag inklusive Umsatzsteuer?'},
  {'answer': 'Am 29.07.2030', 'question': 'Wann wurde das Angebot erstellt?'}]}